Requirements

1) Memory store
2) Chatbot

Now what  Chatbot can do ?

1) The main responsibility of chatbot is to find all the details by asking questions from user, but that questions should be in form of conversation with agent. (SHORT Term Memory is also Implemented)

Now Long Term Memory

1) It hold user name, the final prefereces, each and everything


In [ ]:
import os
import uuid
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_agent
from langchain.agents import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from pydantic import BaseModel, Field
from langchain.tools import tool, ToolRuntime
from langchain.messages import ToolMessage
from langgraph.types import Command
from typing import List, Dict, Optional
from datetime import datetime

from typing import Literal

from langchain.agents import AgentState, create_agent
from langchain.messages import AIMessage, ToolMessage
from langchain.tools import tool, ToolRuntime
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing_extensions import NotRequired

In [ ]:
# API and environments
os.environ["GOOGLE_API_KEY"] = ""
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0, max_retries = 3)
checkpointer = InMemorySaver()

In [3]:
s = datetime.now().isoformat("T")

In [4]:
s.split("T")[0]

'2026-01-12'

In [5]:
# Manual Database
class UserDetails(BaseModel):
    UserName: Optional[str]
    UserID: str
    DateCreated: Optional[str]
    Preferences: Dict
    Email: Optional[str]

class StoreMemory():
    def __init__(self):
        self.users = {}
        self.trip_plans = {}
    def add_or_get_user(self, user_id: str = None, user_name = None, preferences = None, email: str = None):
        if preferences is None:
            preferences = {}
        if  user_id in self.users:
            return self.users[user_id]
        else:
            now = datetime.now().isoformat()
            user_details = UserDetails(
                UserName = user_name,
                UserID = user_id,
                DateCreated = now,
                Preferences = preferences,
                Email = email
            )
            self.users[user_id] = user_details
            return user_details

    def add_travel_details(self, user_id, Departure: str, Destination: List[str], StartDate: str, Duration: str, Budget: str, Interest: List[str], ExtraDetail: List[str], AllDetails: bool):
        trip_id = str(uuid.uuid4())
        trip_id = str(Destination) + trip_id
        trip_info = {
            "user_id": user_id,
            "trip_id": trip_id,
            "Departure": Departure,
            "Destination":Destination,
            "StartDate": StartDate,
            "Duration": Duration, 
            "Budget": Budget,
            "Interest": Interest,
            "ExtraDetail": ExtraDetail,
            "AllDetails": AllDetails
        }
        self.trip_plans[trip_id] = trip_info

    def get_trip_details(self, user_id, specific_trip = False):
        if specific_trip == False:
            trip_details = [trips  for trips in self.trip_plans.values() if trips["user_id"] == user_id]
            return trip_details

local_database = StoreMemory()

In [6]:
local_database.add_or_get_user(user_id = "abc").UserName = "Ahmed"

In [7]:
class TripRequirement(BaseModel):
    Departure: str = None
    Destination: List['str'] = None
    StartDate: str = None
    Duration: str = None
    Budget: str = None
    Interest: List[str] = None
    AllDetails: bool = False
    ExtraDetail: List[str] = None

    def is_complete(self) -> bool:
        return all([self.Destination, self.StartDate, self.Duration, self.Budget, self.Interest, self.AllDetails, self.ExtraDetail])


class ConversationFormat(BaseModel):
    Departure: str = None
    Destination: List['str'] = Field(None, description = "The place you want to go")
    StartDate: str = Field(None, description = "The date to start the trip: YYYY-MM-DD")
    Duration: str = Field(None, description = "The total number of days planned for trip")
    Budget: str = Field(None, description = "The overall budget of the trip")
    Interest: List[str] = Field(None, description = "The personality of person, Reccomendations are givenm based on the Interest of person")
    AllDetails: bool = Field(False, description = "True, if all details are completed, otherwise False")
    ExtraDetail: List[str] = Field(None, description = "Some Extra Information to provide")
    Response: str = Field(None, description = "The response of Model")

class CustomState(AgentState):
    Departure: str = None
    Destination: List['str'] = Field(None, description = "The place you want to go")
    StartDate: str = Field(None, description = "The date to start the trip: YYYY-MM-DD")
    Duration: str = Field(None, description = "The total number of days planned for trip")
    Budget: str = Field(None, description = "The overall budget of the trip")
    Interest: List[str] = Field(None, description = "The personality of person, Reccomendations are givenm based on the Interest of person")
    AllDetails: bool = Field(False, description = "True, if all details are completed, otherwise False")
    ExtraDetail: List[str] = Field(None, description = "Some Extra Information to provide")
    Response: str = Field(None, description = "The response of Model")
    move_to_info_chatbot: bool = Field(False, description = "")


In [8]:
@tool
def time_tool():
        "This Tool is used when you to want to know about the current year, month, and date"
        time = datetime.now().isoformat()
        return time.split("T")[0]

In [10]:
class InformationGatherChatbot():
    def __init__(self, model, response_formatting, threadid, user_id, checkpointer = checkpointer, trip_requirement = TripRequirement(), store = local_database):
        self.model = model
        self.store = store
        self.user_id = user_id
        self.threadid = threadid
        self.response_formatting = response_formatting
        self.checkpointer = checkpointer
        self.trip_requirement = trip_requirement
        self.conversation_prompt = self.prompt()
        self.agent = self.initialize_agent(self.model, self.conversation_prompt, self.checkpointer, self.response_formatting )


    def initialize_agent(self, model, prompt, checkpoint, response_format):
        chatbot = create_agent(model = model, system_prompt = prompt, checkpointer = checkpoint, response_format = response_format, tools = [time_tool])
        return chatbot
    def prompt(self):
        details = self.store.add_or_get_user(user_id =self.user_id)
        if details.UserName == None:

            conversation_system_prompt = """You are a friendly Travel Planning Assistant. Your role is to gather information from the user to create a personalized trip plan.
                    INFORMATION NEEDED:
                    1: Departure: Where are you right now ? 
                    2. Destination: Where do they want to go?
                    3. StartDate: When do they plan to travel? (format: YYYY-MM-DD)
                    4. Duration: How many days? (integer)
                    5. Budget: What's their budget in USD? (number)
                    6. Interests: What are their travel interests? (list: adventure, culture, food, beaches, history, shopping, nature, nightlife, etc.)
                    7. ExtraDetail: if there is anything else you want to add which we have to take care: (List: 5 star hotel, halal food etc). if user says no add "Nothing"
                    8  AllDetails: (Bool) True if user confirm all details and say to give final report.

                    Tools Available:
                    1. time_tool: This tool is used when user don't give you time and ask you to find yourself

                    CONVERSATION RULES:
                    - You have to fill up 7 things, Destination, StartDate, Duration, Budget, Interest, ExtraDetail, AllDetails, Response. In Response the natural and friendly conversation will come which other person will see. for other parameters, if information is not given by user, keep it None and once it gave then update it
                    - Be conversational and friendly.
                    - Reference the conversation history to avoid asking the same question twice.
                    - If the user provides multiple pieces of info at once, confirm them all.
                    - Use the extracted information to ask follow-up questions naturally.
                    -  After the user gave all information, finally ask if provided information is correct and if he says yes, then make all_details as True

                    """
            return conversation_system_prompt
        else:
                conversation_system_prompt = f"""You are a friendly Travel Planning Assistant. Your role is to gather information from the user to create a personalized trip plan.
            The name of person is {details.UserName}
                    INFORMATION NEEDED:
                    1: Departure: Where are you right now ? 
                    2. Destination: Where do they want to go?
                    3. StartDate: When do they plan to travel? (format: YYYY-MM-DD)
                    4. Duration: How many days? (integer)
                    5. Budget: What's their budget in USD? (number)
                    6. Interests: What are their travel interests? (list: adventure, culture, food, beaches, history, shopping, nature, nightlife, etc.)
                    7. ExtraDetail: if there is anything else you want to add which we have to take care: (List: 5 star hotel, halal food etc). if user says no add "Nothing"
                    8  AllDetails: (Bool) True if user confirm all details and say to give final report.
                    
                    Tools Available:
                    1. time_tool: This tool is used when user don't give you time and ask you to find yourself

                    CONVERSATION RULES:
                    - You have to fill up 7 things, Destination, StartDate, Duration, Budget, Interest, ExtraDetail, AllDetails, Response. In Response the natural and friendly conversation will come which other person will see. for other parameters, if information is not given by user, keep it None and once it gave then update it
                    - Be conversational and friendly.
                    - Reference the conversation history to avoid asking the same question twice.
                    - If the user provides multiple pieces of info at once, confirm them all.
                    - Use the extracted information to ask follow-up questions naturally.
                    -  After the user gave all information, finally ask if provided information is correct and if he says yes, then make all_details as True

                    """
                return conversation_system_prompt

    def ask(self, question: str, config_id: str = "1") -> str:
        configuration = {"configurable":{"thread_id":self.threadid}}
        response = self.agent.invoke({"messages":[{"role": "user", "content": question}]}, config = configuration)
        
        Departure = response.get("structured_response").Departure
        Destination = response.get("structured_response").Destination
        StartDate = response.get("structured_response").StartDate
        Duration = response.get("structured_response").Duration
        Budget = response.get("structured_response").Budget
        Interest = response.get("structured_response").Interest
        Response = response.get("structured_response").Response
        ExtraDetail = response.get("structured_response").ExtraDetail
        AllDetails = response.get("structured_response").AllDetails

        return Departure, Destination, StartDate, Duration, Budget, Interest, ExtraDetail, AllDetails, Response

c = InformationGatherChatbot(model, ConversationFormat, user_id = "abc", threadid = "abc123")

In [ ]:
def gather_info_node(state: CustomState):
    messages = state.get("messages", [])
    last_message = messages[-1]
    if hasattr(last_message, 'content'):
        user_input = last_message.content
    else:
        user_input = last_message['content']
    Departure, Destination, StartDate, Duration, Budget, Interest, ExtraDetail, AllDetails, Response = c.ask(user_input)
    return {"Departure":Departure, "Destination":Destination, "StartDate":StartDate, "Duration":Duration, "Budget":Budget, "Interest":Interest, "ExtraDetail":ExtraDetail, "AllDetails":AllDetails, "messages": Response}


def conditional_move(state: CustomState):
    all_con = state.get("AllDetails", None)
    if all_con == True:
        state['move_to_info_chatbot'] = True        
        return "END"
    return "END"

In [13]:
graph = StateGraph(CustomState)
graph.add_node("gather_info_node", gather_info_node)
graph.add_edge(START, "gather_info_node")
graph.add_conditional_edges("gather_info_node", conditional_move, {"gather_info_node": "gather_info_node", "END": END})
builder = graph.compile()

In [16]:
builder.invoke({
        "messages": [
            {
                "role": "user",
                "content": "I'm planning for 5 days",
            }
        ]
    }
)

{'messages': [HumanMessage(content="I'm planning for 5 days", additional_kwargs={}, response_metadata={}, id='c5cf7402-1a1d-45e3-87e2-96b3c9ef2175'),
  HumanMessage(content="Got it! A 5-day trip to Thailand starting on February 12, 2026. To make sure I find the best options for you, what's your estimated budget for this trip in USD?", additional_kwargs={}, response_metadata={}, id='a2146a3d-f6a7-43b7-b1ec-6b32a5a9da8f')],
 'Departure': None,
 'Destination': ['Thailand'],
 'StartDate': '2026-02-12',
 'Duration': '5',
 'Budget': None,
 'Interest': None,
 'AllDetails': False,
 'ExtraDetail': None}

In [14]:
gather_info_node("Hello")

{'Departure': 'None',
 'Destination': [],
 'StartDate': 'None',
 'Duration': 'None',
 'Budget': 'None',
 'Interest': [],
 'ExtraDetail': [],
 'AllDetails': False,
 'messages': "Hello Ahmed! I'm ready to help you plan your next adventure. To get started, could you please tell me where you'll be departing from?"}

In [ ]:
    def check_and_fill_details(self, Departure, Destination, StartDate, Duration, Budget, Interest, ExtraDetail, AllDetails):
        self.trip_requirement.Departure = Departure
        self.trip_requirement.Destination = Destination
        self.trip_requirement.StartDate = StartDate
        self.trip_requirement.Duration = Duration
        self.trip_requirement.Budget = Budget
        self.trip_requirement.Interest = Interest
        self.trip_requirement.ExtraDetail = ExtraDetail
        self.trip_requirement.AllDetails = AllDetails
        
        if self.trip_requirement.is_complete(): 
            self.store.add_travel_details(self.user_id, Departure, Destination, StartDate, Duration, Budget, Interest, ExtraDetail, AllDetails)
            print("Great!! Now Running the Travel Partner to give detailed response. Be Patient...")
            Report = "Here Is your report"
            return True, Report
        Report = ""
        return False, Report

In [ ]:
print(c.ask("Hello"))


In [ ]:
print(c.ask("I'm planning to go to Thailand"))

In [ ]:
print(c.ask("Hmmm I'm not sure, what you suggest when do I Have to visit ?"))

In [ ]:
print(c.ask("Make it November 2026 then, and any date is fine whatever you think"))

In [ ]:
print(c.ask("I think one week is more than enough for me"))

In [ ]:
print(c.ask(" I think 7 days are not enough change t 10"))

In [ ]:
print(c.ask("oooh!!! that's a though one, I'm not sure how much budget is good for this trip ?"))

In [ ]:
print(c.ask("Make it 1000"))

In [ ]:
print(c.ask("I'm into Parties, beaches"))

In [ ]:
print(c.ask("I think I want to go to China not Thailand, so change the country, rest is perfect"))

In [ ]:
print(c.ask("Yes, One more things add halal food option "))

In [ ]:
print(c.ask("Pakistan"))

In [ ]:
print(c.ask("I will start my trip on january 20th 2026"))

In [ ]:
print(c.ask("Yes"))

In [ ]:
local_database.get_trip_details("abc")

In [ ]:
# conversation_system_prompt = """You are a friendly Travel Planning Assistant. Your role is to gather information from the user to create a personalized trip plan.

# INFORMATION NEEDED:
# 1. Destination: Where do they want to go?
# 2. Start Date: When do they plan to travel? (format: YYYY-MM-DD)
# 3. Duration: How many days? (integer)
# 4. Budget: What's their budget in USD? (number)
# 5. Interests: What are their travel interests? (list: adventure, culture, food, beaches, history, shopping, nature, nightlife, etc.)
# 6. Extra_detail: if there is anything else you want to add which we have to take care: (List: 5 star hotel, halal food etc)
# 7 all_details: (Bool) True if user confirm all details and say to give final report.
# CONVERSATION RULES:
# - You have to fill up 6 things, Destination, Start_Date, Duration, Budget, Interest, and Response. In Response the natural and friendly conversation will come which other person will see. for other parameters, if information is not given by user, keep it None and once it gave then update it
# - Be conversational and friendly.
# - Reference the conversation history to avoid asking the same question twice.
# - If the user provides multiple pieces of info at once, confirm them all.
# - Use the extracted information to ask follow-up questions naturally.
# -  After the user gave all information, finally ask if provided information is correct and if he says yes, then make all_details as True

# """
# chatbot = create_agent(model = model, system_prompt = conversation_system_prompt, checkpointer = checkpointer, response_format = ConversationFormat)

# def ask(question: str, config_id: str = "1"):
#     configuration = {"configurable":{"thread_id":config_id}}
#     response = chatbot.invoke({"messages":[{"role": "user", "content": question}]}, config = configuration)
    
#     Destination = response.get("structured_response").Destination
#     Start_Date = response.get("structured_response").Start_Date
#     Duration = response.get("structured_response").Duration
#     Budget = response.get("structured_response").Budget
#     Interest = response.get("structured_response").Interest
#     Response = response.get("structured_response").Response
#     #return Destination, Start_Date, Duration, Budget, Interest, Response
#     return f"""
#         Destination: {Destination}, Start_Date: {Start_Date}, Duration: {Duration}, Budget: {Budget}, Interest: {Interest} 
#         Response: {Response}
#         """